In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta

In [9]:
data = pd.read_csv("../../../data/aapl_1h_train.csv")
tsi_data = ta.momentum.TSIIndicator(data.Close, window_slow = 25, window_fast = 13)
data["TSI"] = tsi_data.tsi()
data = data.dropna()

In [10]:
data

,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,TSI
37,1603121400,0,2020-10-19 15:30:00,119.110000,119.510002,118.400398,118.815002,11743873.0,8.185917
38,1603125000,0,2020-10-19 16:30:00,118.800003,119.139999,118.584999,118.699996,8185217.0,6.227720
39,1603128600,0,2020-10-19 17:30:00,118.699996,118.750000,117.529998,117.830497,13320284.0,2.988857
40,1603132200,0,2020-10-19 18:30:00,117.690002,117.830001,116.160003,116.363700,20231178.0,-2.263912
41,1603135800,0,2020-10-19 19:30:00,116.349998,116.900001,115.660003,116.000000,15044773.0,-7.058797
...,...,...,...,...,...,...,...,...,...
390,1609432200,0,2020-12-31 16:30:00,132.529998,132.800003,131.720001,132.619995,13538948.0,-1.705551
391,1609435800,0,2020-12-31 17:30:00,132.619995,132.717605,132.310104,132.560806,7474176.0,-4.240807
392,1609439400,0,2020-12-31 18:30:00,132.565002,132.889999,132.009994,132.764999,9408857.0,-5.929122
393,1609443000,0,2020-12-31 19:30:00,132.770004,133.179992,132.565994,132.800003,10147999.0,-7.278637


In [11]:
def def_trading_strategy(data, n_shares, stop_loss_close, take_profit_close, Umbral):
    cash= 1_000_000
    com = 0.125/100
    history = []
    portfolio_value = []
    active_operations = []
    total_profit_loss = 0

    for i, row in data.iterrows():
        # Close active operation
        active_op_temp = []
        for operation in active_operations:
            if operation["stop_loss"] > row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
                # Calcular la ganancia o pérdida y actualizar el total
                profit_loss = (row.Close * operation["n_shares"]) * (1 - com) - (operation["bought"] * operation["n_shares"])
                total_profit_loss += profit_loss
                history.append({"timestamp": row.Timestamp, "profit_loss": profit_loss})
            elif operation["take_profit"] < row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
                # Calcular la ganancia o pérdida y actualizar el total
                profit_loss = (row.Close * operation["n_shares"]) * (1 - com) - (operation["bought"] * operation["n_shares"])
                total_profit_loss += profit_loss
                history.append({"timestamp": row.Timestamp, "profit_loss": profit_loss})
            else:
                active_op_temp.append(operation)
        active_operations = active_op_temp
            
        # ¿Tenemos suficiente efectivo?
        if cash < (row.Close * n_shares * (1 + com)):
            asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
            portfolio_value.append(cash + asset_vals)
            continue
        
        # Analizar la señal larga
        if row.TSI <= Umbral:  # If ceil(Op)
            active_operations.append({
                "timestamp": row.Timestamp,
                "bought": row.Close,
                "n_shares": n_shares,
                "type": "long",
                "stop_loss": row.Close * stop_loss_close,
                "take_profit": row.Close * take_profit_close
            })
            cash -= row.Close * n_shares * (1 + com)
        
        asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
        portfolio_value.append(cash + asset_vals)
    
    return total_profit_loss

In [12]:
import optuna

def objective(trial):
    # Define los rangos de búsqueda para los parámetros
    n_shares = trial.suggest_int('n_shares', 1, 50)
    stop_loss_close = trial.suggest_uniform('stop_loss_close', 0.80, 1)
    take_profit_close = trial.suggest_uniform('take_profit_close', 1, 2)
    Umbral = trial.suggest_uniform('Umbral', -40, 10)

    # Evalúa la función de trading con los parámetros sugeridos
    profit_loss = def_trading_strategy(data, n_shares, stop_loss_close, take_profit_close, Umbral)
    
    return profit_loss  # Devuelve la métrica que se debe minimizar/maximizar

# Crea un estudio Optuna
study = optuna.create_study(direction='maximize')  # 'maximize' si se quiere maximizar la métrica

# Ejecuta la optimización
study.optimize(objective, n_trials=20)  # Puedes ajustar n_trials según tus recursos y tiempo disponible

# Obtiene los mejores parámetros encontrados
best_params = study.best_params
best_profit_loss = study.best_value

print("Mejores parámetros encontrados:")
print(best_params)
print("Rendimiento (Profit/Loss):", best_profit_loss)


[I 2024-02-25 22:59:21,862] A new study created in memory with name: no-name-2b5451ff-94cd-403c-9a7d-b7b833be4e15
C:\Users\carlo\AppData\Local\Temp\ipykernel_29108\260268852.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  stop_loss_close = trial.suggest_uniform('stop_loss_close', 0.80, 1)
C:\Users\carlo\AppData\Local\Temp\ipykernel_29108\260268852.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  take_profit_close = trial.suggest_uniform('take_profit_close', 1, 2)
C:\Users\carlo\AppData\Local\Temp\ipykernel_29108\260268852.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_f

Mejores parámetros encontrados:
{'n_shares': 10, 'stop_loss_close': 0.8791099117690606, 'take_profit_close': 1.1535552563836566, 'Umbral': 0.5043550157272385}
Rendimiento (Profit/Loss): 23259.318356325013


In [13]:
# Condiciones para comprar y vender
condicion_compra = data["TSI"] <= -32
condicion_venta = data["TSI"] > 30

# Aplicando las condiciones para obtener señales de compra y venta
data['Senal_TSI'] = ''
data.loc[condicion_compra, 'Senal_TSI'] = 'Compra'
data.loc[condicion_venta, 'Senal_TSI'] = 'Venta'
data.to_excel('../../../data/1h/TSI_BOS.xlsx', index=False)

In [14]:
data['Senal_TSI'].value_counts()

Senal_TSI
          287
Venta      45
Compra     11
Name: count, dtype: int64